# PROV API dashboard

In [6]:
from IPython.display import HTML
import requests
import pandas as pd
import altair as alt

In [8]:
api_url = "https://api.prov.vic.gov.au/search/query"
params = {
    "q": "*:*",
}
response = requests.get(api_url, params=params)
data = response.json()

display(HTML(f"The PROV API currently returns {data['response']['numFound']:,} results."))

In [4]:
def get_facets(field):
    url = f"https://api.prov.vic.gov.au/search/query?wt=json&q=*:*&facet=true&facet.field={field}&rows=0"
    response = requests.get(url)
    data = response.json()
    values = data["facet_counts"]["facet_fields"][field]
    return [{"facet": values[i], "count": values[i+1]} for i in range(0, len(values), 2)]

def make_facet_chart(field):
    facets = get_facets(field)
    df = pd.DataFrame(facets)
    chart = alt.Chart(df).mark_bar().encode(
        x=alt.X("facet:N", title=None),
        y=alt.Y("count:Q"),
        color=alt.Color("facet:N", title=field, legend=None),
        tooltip = [alt.Tooltip("facet", title=field), alt.Tooltip("count", format=",")]
    ).properties(width=200, height=200, title=field)
    return chart

In [5]:
charts = []
for field in ["category", "entity", "record_form", "format", "location", "rights_status"]:
   charts.append(make_facet_chart(field))
display(alt.concat(*charts, columns=3).properties(padding=20))

alt.ConcatChart(...)